# Train a local segmentation model for human faces and deploy to Amazon SageMaker with a custom Docker container

Useful commands
* `conda env create -f local_env.yml`
* `sudo ln -s /home/ubuntu/gumroad_sagemaker_docker_inference/gumroad_sagemaker/sagemaker_deploy/opt/ml/ /opt/ml`
* `docker run --name gumserve --rm -it gumroad serve`

## Train model

### Data

In [ ]:
from fastai.vision.all import *

In [ ]:
path = Path("/home/ubuntu/data/faces_msft_1000")

In [ ]:
def binarize_image(path: Union[str, Path], overwrite: bool = False) -> None:
    if overwrite:
        o = np.array(Image.open(str(path).replace(".png", "_seg.png")))
        bin = np.ones(o.shape) * 255
        bin[(o==0) | (o==12) | (o==13) | (o==15) | (o==17) | (o==255)] = 0
        Image.fromarray(bin.astype("uint8")).convert("L").save(str(path).replace(".png", "_bin.png"))

In [ ]:
image_files = get_image_files(path)
image_files = [f for f in image_files if "seg" not in str(f) and "bin" not in str(f)]
for image_file in image_files: binarize_image(image_file, overwrite=False)
len(image_files)

In [ ]:
get_images = lambda p: L(x for x in get_image_files(p) if x.stem.isnumeric())
image_files = get_images(path)
image_files

In [ ]:
codes = ["background", "face"]
name2id = {v:k for k,v in enumerate(codes)}
name2id

In [ ]:
img_fn = image_files[10]
img = PILImage.create(img_fn)
img.show(figsize=(5,5))

In [ ]:
def get_msk(fn, p2c):
  fn = path/f'{fn.stem}_bin{fn.suffix}'
  msk = np.array(PILMask.create(fn))
  for i, val in enumerate(p2c):
    msk[msk==p2c[i]] = val
  return PILMask.create(msk)

p2c = {0: 0, 1: 255}

In [ ]:
msk = PILMask.create(get_msk(img_fn, p2c))
msk.show(figsize=(5,5), alpha=1)

In [ ]:
ctx = img.show()
msk.show(ctx=ctx)

In [ ]:
sz = msk.shape; sz

### Model

In [ ]:
def get_y(o): return get_msk(o, p2c)

faces = DataBlock(blocks=(ImageBlock, MaskBlock(codes)),
                  get_items=get_images,
                  splitter=RandomSplitter(),
                  get_y=get_y,
                  batch_tfms=[*aug_transforms(size=sz), Normalize.from_stats(*imagenet_stats)])

In [ ]:
dls = faces.dataloaders(path, bs=8)

In [ ]:
dls.show_batch(max_n=4, vmin=0, vmax=1, figsize=(10,10))

In [ ]:
get_c(dls)

In [ ]:
x, y = dls.one_batch()
x.shape, y.shape

In [ ]:
dls.vocab = codes

In [ ]:
void_code = name2id['background']; void_code

In [ ]:
def acc_faces(inp, targ):
  targ = targ.squeeze(1)
  mask = targ != void_code
  return (inp.argmax(dim=1)[mask]==targ[mask]).float().mean()

In [ ]:
opt_func = partial(Adam, lr=1e-3, wd=0.01)

learn = unet_learner(dls, resnet34, loss_func=CrossEntropyLossFlat(axis=1), opt_func=opt_func, metrics=acc_faces, norm_type=None, wd_bn_bias=True)

In [ ]:
#learn.summary()

In [ ]:
torch.cuda.is_available()

In [ ]:
learn.fine_tune(4, base_lr=1e-3, freeze_epochs=1)

In [ ]:
learn.export('faces.pkl')

In [ ]:
learn.recorder.plot_sched()

In [ ]:
learn.show_results(max_n=4, figsize=(6,10))

### Inference

In [ ]:
from fastai.vision.all import *

In [ ]:
def get_y(x): pass
def acc_faces(x): pass

learn = load_learner('faces.pkl', cpu=True)

In [ ]:
img = Image.open("./test_faces/fra2.jpg")
print(img.shape)
img.thumbnail((512, 512))
img = PILImage.create(img)
print(img.shape); img

In [ ]:
pred_mask, _, _ = learn.predict(img); pred_mask.shape

In [ ]:
def resize_mask_to_img(img, pred_mask):
    if pred_mask.max() == 1: pred_mask = pred_mask.mul(255)
    pil_img = Image.fromarray(pred_mask.byte().cpu().numpy())
    h, w = img.shape
    resized_img = pil_img.resize((w, h))
    return TensorMask(resized_img)

In [ ]:
resize_mask_to_img(img, pred_mask).show()

In [ ]:
ctx = img.show()
pred_mask = resize_mask_to_img(img, pred_mask)
pred_mask.show(ctx=ctx)

In [ ]:
def show_pred(img_path, mask):
    img = Image.open(img_path)
    img.thumbnail((512, 512))
    img = PILImage.create(img)
    ctx = img.show()
    mask = resize_mask_to_img(img, mask)
    mask.show(ctx=ctx)

In [ ]:
show_pred("./test_faces/fra2.jpg", pred_mask)

## Testing `serve` script locally

In [ ]:
from fastai.vision.all import *
import requests, io, base64

In [ ]:
def ping(url):
    url = f'http://{url}:8080/ping'
    return requests.get(url)

def post_local(url, img_path):
    img = Image.open(img_path)

    byte_arr = io.BytesIO()
    img.save(byte_arr, format='JPEG')
    encoded_image = base64.encodebytes(byte_arr.getvalue()).decode('ascii')
    url = f'http://{url}:8080/invocations'

    headers = {'Content-Type': 'application/json'}
    data = {'image': encoded_image}

    return requests.post(url, headers=headers, json=data)
    
def post_process_mask(data):
    mask_data = data['mask']
    mask_numpy = np.asarray(mask_data)
    mask_numpy = mask_numpy.reshape(512, 512)

    return TensorMask(mask_numpy)

In [ ]:
img_path = "./test_faces/fra1.jpg"
#url = "172.17.0.2"
url = "localhost"

In [ ]:
response = ping(url)
response.text, response.status_code

In [ ]:
response = post_local(url, img_path)
response.status_code

In [ ]:
data = response.json()
mask = post_process_mask(data)
mask.shape

In [ ]:
show_pred(img_path, mask)

## Deploying to SageMaker

In [ ]:
def resize_mask_to_img(img, pred_mask):
    if pred_mask.max() == 1: pred_mask = pred_mask.mul(255)
    pil_img = Image.fromarray(pred_mask.byte().cpu().numpy())
    h, w = img.shape
    resized_img = pil_img.resize((w, h))
    return TensorMask(resized_img)

def show_pred(img_path, mask):
    img = Image.open(img_path)
    img.thumbnail((512, 512))
    img = PILImage.create(img)
    ctx = img.show()
    mask = resize_mask_to_img(img, mask)
    mask.show(ctx=ctx)
    
def post_process_mask(data):
    mask_data = data['mask']
    mask_numpy = np.asarray(mask_data)
    mask_numpy = mask_numpy.reshape(512, 512)

    return TensorMask(mask_numpy)

In [ ]:
import boto3, sagemaker, json, base64
from fastai.vision.all import *

region = "eu-west-1"
role = "arn:aws:iam::257446244580:role/sagemaker-icevision"
sm_client = boto3.client("sagemaker", region_name=region)
runtime_sm_client = boto3.client("sagemaker-runtime", region_name=region)
session = sagemaker.Session(boto_session=boto3.Session(region_name=region))
ecr_image = "257446244580.dkr.ecr.eu-west-1.amazonaws.com/custom-images:gumroad"
name = "gumroad"

In [ ]:
!cd sagemaker_deploy/opt/ml/model/
!tar -cvpzf model.tar.gz faces.pkl

In [ ]:
default_bucket = session.default_bucket(); default_bucket

In [ ]:
model_uri = session.upload_data(path="/home/ubuntu/KagglePlaygrounds/gumroad_sagemaker/model.tar.gz", key_prefix="gumroad"); model_uri

### Deploy via the `sagemaker` SDK

In [ ]:
model = sagemaker.model.Model(image_uri=ecr_image,
                              name=name,
                              #model_data=model_uri, # REMOVE IF USING Dockerfile.ModelLocal, as model is already embedded in the image
                              role=role,
                              sagemaker_session=session,
                              predictor_cls=sagemaker.Predictor
                             )

predictor = model.deploy(initial_instance_count=1, 
                         instance_type='ml.m5.large', 
                         endpoint_name=name)

### Deploy via the `boto3` client API

In [ ]:
model = sm_client.create_model(
    ModelName=name, 
    ExecutionRoleArn=role, 
    PrimaryContainer={
        "Image": ecr_image,
        "ModelDataUrl": model_uri, # REMOVE IF USING Dockerfile.ModelLocal, as model is already embedded in the image
    }
)

endpoint_config = sm_client.create_endpoint_config(
    EndpointConfigName=name,
    ProductionVariants=[
        {
            "InstanceType": 'ml.m5.large',
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": name,
            "VariantName": "AllTraffic",
        }
    ],
)

endpoint = sm_client.create_endpoint(
    EndpointName=name, EndpointConfigName=name
)

def wait_for_endpoint_in_service(endpoint_name):
    print("Waiting for endpoint in service")
    while True:
        details = sm_client.describe_endpoint(EndpointName=endpoint_name)
        status = details["EndpointStatus"]
        if status in ["InService", "Failed"]:
            print("\nDone!")
            break
        print(".", end="", flush=True)
        time.sleep(30)


wait_for_endpoint_in_service(name)

sm_client.describe_endpoint(EndpointName=name)

### Invoke the endpoint

In [ ]:
predictor = sagemaker.Predictor(endpoint_name=name, sagemaker_session=session); predictor.endpoint_name

In [ ]:
img_path = "./test_faces/fra1.jpg"

with open(img_path, "rb") as img_file:
    b64_string = base64.b64encode(img_file.read())

In [ ]:
response = predictor.predict('{"image": "'+b64_string.decode('utf-8')+'"}', initial_args={"ContentType": "application/json"})

In [ ]:
data = json.loads(response)
mask = post_process_mask(data)
mask.shape

In [ ]:
show_pred(img_path, mask)

In [ ]:
response = runtime_sm_client.invoke_endpoint(EndpointName=name, ContentType="application/json", Body='{"image": "'+b64_string.decode('utf-8')+'"}')

In [ ]:
response = json.loads(response["Body"].read().decode("utf-8"))
mask = post_process_mask(data)
mask.shape

In [ ]:
show_pred(img_path, mask)

In [ ]:
sm_client.delete_model(ModelName=name)
sm_client.delete_endpoint_config(EndpointConfigName=name)
sm_client.delete_endpoint(EndpointName=name)